# Structure of This Notebook:
1. Data import and exploration
2. Data Preprocessing
    1. Missing data
    2. Outliers
    3. Data type Correction
    4. Label encoding
    5. Skewness
3. Feature Engineering 
    * New variables Generation
4. Modelling
    * Lasso regression
    * Ridge regression
    * Net Elastic Regression
5. Peformance validation
    * Train/Validation Split
    * Cross-validation     

In [ ]:
##Team Assignment1##
library(plyr)
library(tidyverse)
library(ggplot2)
library(corrplot)
library(glmnet)
library(caTools)
library(mice)
library(ggcorrplot)
library(caret)
library(tidyverse)
library(Metrics)
library(corrplot)
library(moments)
library(scales)
library(dplyr)
library(caret)
library(Hmisc)
library(glmnet)
library(naniar)

In [ ]:
#Importing the data
train<- read.csv("../input/house-prices-advanced-regression-techniques/train.csv")
test<- read.csv("../input/house-prices-advanced-regression-techniques/test.csv")

#test.data has 1459 obs of 80 variables
#train.data has 1460 obs of 81 variables

#Observing the structure of both data sets
str(train[,0:5])
str(test[,0:5])
dim(train)
dim(test)

In [ ]:
head(train)
head(test)

In [ ]:
#merging both test & train datasets to one dataset
test$SalePrice <- NA
full<- rbind(train, test)

In [ ]:
#identify column names in one dataset but not in other
#A good sign if we see none.
setdiff(names(train), names((test)))

# Dealing with Missing data

Full dataset has 80 variables and some may have lots of missing records.

Approaches:
1. Review columns with high missing rates.
2. Impute numerical values with their medians
3. Impute categorical values with their most frequent values

In [ ]:
#Check the missing rate from the highest to the lowest
NA_col=which(colSums(is.na(full))>0)

round(sort(colSums(sapply(full[NA_col], is.na))/dim(full)[1], decreasing=TRUE),3)

In [ ]:
#There are many variables that are more suitable to be nominal variables
Qualities <- c('None' = 0, 'Po' = 1, 'Fa' = 2, 'TA' = 3, 'Gd' = 4, 'Ex' = 5)
full$PoolQC[is.na(full$PoolQC)] <- 'None'
full$PoolQC<-as.integer(revalue(full$PoolQC, Qualities))

full$MiscFeature[is.na(full$MiscFeature)] <- 'None'
full$MiscFeature <- as.factor(full$MiscFeature)

full$Alley[is.na(full$Alley)] <- 'None'
full$Alley <- as.factor(full$Alley)

full$Fence[is.na(full$Fence)] <- 'None'
full$Fence<- as.factor(full$Fence)

full$FireplaceQu[is.na(full$FireplaceQu)] <- 'None'
full$FireplaceQu<-as.integer(revalue(full$FireplaceQu, Qualities))

full$LotFrontage <- with(full, impute(LotFrontage, median))

full$GarageType[is.na(full$GarageType)] <- 'No Garage'
full$GarageFinish[is.na(full$GarageFinish)] <- 'None'
Finish <- c('None'=0, 'Unf'=1, 'RFn'=2, 'Fin'=3)

full$GarageFinish<-as.integer(revalue(full$GarageFinish, Finish))

full$GarageQual[is.na(full$GarageQual)] <- 'None'
full$GarageQual<-as.integer(revalue(full$GarageQual, Qualities))

full$GarageCond[is.na(full$GarageCond)] <- 'None'
full$GarageCond<-as.integer(revalue(full$GarageCond, Qualities))

full$BsmtFinType2[333] <- names(sort(-table(full$BsmtFinType2)))[1]
full$BsmtExposure[c(949, 1488, 2349)] <- names(sort(-table(full$BsmtExposure)))[1]
full$BsmtCond[c(2041, 2186, 2525)] <- names(sort(-table(full$BsmtCond)))[1]
full$BsmtQual[c(2218, 2219)] <- names(sort(-table(full$BsmtQual)))[1]

full$BsmtQual[is.na(full$BsmtQual)] <- 'None'
full$BsmtQual<-as.integer(revalue(full$BsmtQual, Qualities))

full$BsmtCond[is.na(full$BsmtCond)] <- 'None'
full$BsmtCond<-as.integer(revalue(full$BsmtCond, Qualities))

full$BsmtExposure[is.na(full$BsmtExposure)] <- 'None'
Exposure <- c('None'=0, 'No'=1, 'Mn'=2, 'Av'=3, 'Gd'=4)

full$BsmtExposure<-as.integer(revalue(full$BsmtExposure, Exposure))

full$BsmtFinType1[is.na(full$BsmtFinType1)] <- 'None'
FinType <- c('None'=0, 'Unf'=1, 'LwQ'=2, 'Rec'=3, 'BLQ'=4, 'ALQ'=5, 'GLQ'=6)

full$BsmtFinType1<-as.integer(revalue(full$BsmtFinType1, FinType))

full$BsmtFinType2[is.na(full$BsmtFinType2)] <- 'None'
FinType <- c('None'=0, 'Unf'=1, 'LwQ'=2, 'Rec'=3, 'BLQ'=4, 'ALQ'=5, 'GLQ'=6)

full$BsmtFinType2<-as.integer(revalue(full$BsmtFinType2, FinType))

full$BsmtFullBath[is.na(full$BsmtFullBath)] <-0

full$BsmtHalfBath[is.na(full$BsmtHalfBath)] <-0

full$BsmtFinSF1[is.na(full$BsmtFinSF1)] <-0

full$BsmtFinSF2[is.na(full$BsmtFinSF2)] <-0

full$BsmtUnfSF[is.na(full$BsmtUnfSF)] <-0
full$TotalBsmtSF[is.na(full$TotalBsmtSF)] <-0
full$GarageYrBlt[is.na(full$GarageYrBlt)] <- 0
full$GarageCars[2577] <- 0
full$GarageArea[2577] <- 0

full$MasVnrArea <- with(full, impute(MasVnrArea, median))
full$MasVnrType[is.na(full$MasVnrType)] <- 'None'
full$MSZoning[is.na(full$MSZoning)] <- names(sort(-table(full$MSZoning)))[1]
full$MSZoning <- as.factor(full$MSZoning)

full$KitchenQual[is.na(full$KitchenQual)] <- 'TA' #replace with most common value
full$KitchenQual<-as.integer(revalue(full$KitchenQual, Qualities))
full$Utilities <- NULL
#impute mode 
full$Functional[is.na(full$Functional)] <- names(sort(-table(full$Functional)))[1]

full$Functional <- as.integer(revalue(full$Functional, c('Sal'=0, 'Sev'=1, 'Maj2'=2, 'Maj1'=3, 'Mod'=4, 'Min2'=5, 'Min1'=6, 'Typ'=7)))

full$Exterior1st[is.na(full$Exterior1st)] <- names(sort(-table(full$Exterior1st)))[1]

full$Exterior1st <- as.factor(full$Exterior1st)

full$Exterior2nd[is.na(full$Exterior2nd)] <- names(sort(-table(full$Exterior2nd)))[1]

full$Exterior2nd <- as.factor(full$Exterior2nd)

full$ExterQual<-as.integer(revalue(full$ExterQual, Qualities))
full$ExterCond<-as.integer(revalue(full$ExterCond, Qualities))

full$Electrical[is.na(full$Electrical)] <- names(sort(-table(full$Electrical)))[1]

full$Electrical <- as.factor(full$Electrical)

full$SaleType[is.na(full$SaleType)] <- names(sort(-table(full$SaleType)))[1]
full$SaleType <- as.factor(full$SaleType)

In [ ]:
vis_miss(full)

We have dealt the missing values for all variables other than SalePrice. Let's move to the next step

# Outliers

In [ ]:
ggplot(full, aes(x=GrLivArea, y=SalePrice))+geom_point()

In [ ]:
#there are 2 points that are outliers from the chart
full<- full[!(full$GrLivArea>4500 & !is.na(full$SalePrice)),]

In [ ]:
#GarageYrBlt 2207 looks odd and impossible
keep <- c("GarageYrBlt","YearBuilt","YearRemodAdd")
check<-full[,names(full) %in% keep]
summary(check)

full$GarageYrBlt[full$GarageYrBlt==2207]<- 2007

# Visualization

In [ ]:
# show the histogram of the Sales Price and it's right-skewed,
hist(train$SalePrice)

In [ ]:
# show the histogram of the log(Sales Price), more normally distributed 
hist(log(train$SalePrice))

In [ ]:
#Check the correlations
#select the integer variables
train2 <- which(sapply(train, is.integer))
corr_name=names(train2)

cormat <- round(cor(train[,corr_name]),2)
                
top_var=as.matrix(sort(cormat[,'SalePrice'], decreasing = TRUE))

corr_high=names(which(apply(top_var, 1, function(x) abs(x)>0.5)))
ggcorrplot(cormat[corr_high,corr_high])

# Data type correction

In [ ]:
full$MoSold <- as.factor(full$MoSold)
full$MSSubClass <- as.factor(full$MSSubClass)

# Skewness Transformation

In [ ]:
# this variable is skewed - let's correct this by taking log(x+1)
hist(full$LotFrontage)

In [ ]:
# for numeric feature with excessive skewness, perform log transformation
# Exclude SalePrice
drop<-c("Id","SalePrice")
fullv2<-full[,!(names(full) %in% drop)]


numericVars <- names(which(sapply(fullv2, is.numeric))) 
skewed_feats <- skewness(full[numericVars])

In [ ]:
numericVars

In [ ]:
skewed_feats

In [ ]:
# keep only features that are highly skewed
skewed_feats <- skewed_feats[skewed_feats > 0.75]
skewed_feats

In [ ]:
# transform skewed features with log(x + 1)
for(x in names(skewed_feats)) {
  full[[x]] <- log(full[[x]] + 1)}

In [ ]:
#This looks much better now!
hist(full$LotFrontage)

# Feature Engineering

we are going to buil many new varaibles that might help us to make the predictions

In [ ]:
full$TotBathrooms <- full$FullBath + (full$HalfBath*0.5) + full$BsmtFullBath + (full$BsmtHalfBath*0.5)
full$Remod <- ifelse(full$YearBuilt==full$YearRemodAdd, 0, 1) #0=No Remodeling, 1=Remodeling
full$Age <- as.numeric(full$YrSold)-full$YearRemodAdd
#full$HasDeck <- ifelse(full$WoodDeckSF > 0,'Yes','No')

In [ ]:
full$NeighRich[full$Neighborhood %in% c('StoneBr', 'NridgHt', 'NoRidge')] <- 2
full$NeighRich[!full$Neighborhood %in% c('MeadowV', 'IDOTRR', 'BrDale', 'StoneBr', 'NridgHt', 'NoRidge')] <- 1
full$NeighRich[full$Neighborhood %in% c('MeadowV', 'IDOTRR', 'BrDale')] <- 0
full$TotalSqFeet <- full$GrLivArea + full$TotalBsmtSF
full$TotalPorchSF <- full$OpenPorchSF + full$EnclosedPorch + full$X3SsnPorch + full$ScreenPorch

#Interactions
full$NeighRich_size<-full$NeighRich*full$TotalSqFeet
full$NeighRich_age<-full$Age*full$NeighRich
full$NeighRich_porch <- full$NeighRich*full$TotalPorchSF
full$NeighRich_Quality<- full$NeighRich*full$OverallQual  
full$NeighRich_Cond<- full$NeighRich*full$OverallCond  
full$NeighRich_bath<- full$NeighRich*full$TotBathrooms
full$NeighRich_remod<-full$NeighRich*full$Remod

In [ ]:
# one-hot encoding
drop<-c("Id","SalePrice")
data<-full[,!(names(full) %in% drop)]

combined <- as.data.frame(model.matrix(~.-1, data))
dim(combined)

In [ ]:
combined<-cbind(combined, full['SalePrice'])

In [ ]:
#split data on SalePrice - NA vs Non-NA
train1 <- combined[!is.na(full$SalePrice),]
test1 <- combined[is.na(full$SalePrice),]

# Model Development and Performance evaluation
Train1 have to be splitted into training and validation (90%&10%). This ensures more data to train, and enough data to check which modelling method is optimum fit

In [ ]:
set.seed(123123)
inx <- sample.split(seq_len(nrow(train1)), 0.9) 
training <- train1[inx, ] # Training set
validation <-  train1[!inx, ] # The rest goes to the testing set

In [ ]:
head(training)
dim(training)
dim(validation)

In [ ]:
y_training<-log(training$SalePrice+1)
training2<-training[,!(names(training) %in% drop)]
X<-model.matrix(~., training2)[,-1]

In [ ]:
#Selecting the best penalty lambda
# lasso
crossval_lasso <-  cv.glmnet(x = X, y = y_training, alpha = 1) #create cross-validation data. By default, the function performs ten-fold cross-validation, though this can be changed using the argument nfolds. 
plot(crossval_lasso)

# Ridge
crossval_ridge <-  cv.glmnet(x = X, y = y_training, alpha = 0)
plot(crossval_ridge)

In [ ]:
penalty_lasso <- crossval_lasso$lambda.min
penalty_ridge <- crossval_ridge$lambda.min

#below can be spotted on the plot
log(penalty_lasso)
log(penalty_ridge)

In [ ]:
lasso_opt_fit <-glmnet(x = X, y = y_training, alpha = 1, lambda = penalty_lasso) #estimate the model with the optimal penalty
ridge_opt_fit <-glmnet(x = X, y = y_training, alpha = 0, lambda = penalty_ridge)

In [ ]:
coef <- data.frame(coef.name = dimnames(coef(lasso_opt_fit,s=lasso_opt_fit$bestTune$lambda))[[1]], 
           coef.value = matrix(coef(lasso_opt_fit,s=lasso_opt_fit$bestTune$lambda)))

coef <- coef[-1,]

picked_features <- nrow(filter(coef,coef.value!=0))
not_picked_features <- nrow(filter(coef,coef.value==0))

cat("Lasso picked",picked_features,"variables and eliminated the other",
    not_picked_features,"variables\n")

In [ ]:
# sort coefficients in ascending order
coef <- arrange(coef,-coef.value)

# extract the top 10 and bottom 10 features
imp_coef <- rbind(head(coef,10),
                  tail(coef,10))

ggplot(imp_coef) +
    geom_bar(aes(x=reorder(coef.name,coef.value),y=coef.value),
             stat="identity") +
    ylim(-0.5,0.5) +
    coord_flip() +
    ggtitle("Coefficents in the Lasso Model") +
    theme(axis.title=element_blank())

In [ ]:
coef <- data.frame(coef.name = dimnames(coef(ridge_opt_fit,s=ridge_opt_fit$bestTune$lambda))[[1]], 
           coef.value = matrix(coef(ridge_opt_fit,s=ridge_opt_fit$bestTune$lambda)))

coef <- coef[-1,]

picked_features <- nrow(filter(coef,coef.value!=0))
not_picked_features <- nrow(filter(coef,coef.value==0))

cat("Ridge picked",picked_features,"variables and eliminated the other",
    not_picked_features,"variables\n")

In [ ]:
# sort coefficients in ascending order
coef <- arrange(coef,-coef.value)

# extract the top 10 and bottom 10 features
imp_coef <- rbind(head(coef,10),
                  tail(coef,10))

ggplot(imp_coef) +
    geom_bar(aes(x=reorder(coef.name,coef.value),y=coef.value),
             stat="identity") +
    ylim(-0.5,0.5) +
    coord_flip() +
    ggtitle("Coefficents in the Ridge Model") +
    theme(axis.title=element_blank())

In [ ]:
# Predict on validation dataset
validation2<-validation[,!(names(validation) %in% drop)]
newX<-model.matrix(~., validation2)[,-1]
lasso_val <- exp(predict(lasso_opt_fit, s = penalty_lasso, newx =newX))
ridge_val <- exp(predict(ridge_opt_fit, s = penalty_ridge, newx =newX))

# MSE, RMSE and MAPE

In [ ]:
lasso.val.MSE <- mean((lasso_val-validation$SalePrice)^2) #calculate and display MSE in the testing set
lasso_val_RMSE <- rmse(validation$SalePrice, lasso_val) #calculate and display RMSE - Root Mean Squared Error
lasso_val_MAPE <- mean(abs(lasso_val-validation$SalePrice)/validation$SalePrice*100) # MAPE: mean absolute percentage error 

ridge.val.MSE <- mean((ridge_val-validation$SalePrice)^2) #calculate and display MSE in the testing set
ridge_val_RMSE <- rmse(validation$SalePrice, ridge_val) #calculate and display RMSE 
ridge_val_MAPE <-mean(abs(ridge_val-validation$SalePrice)/validation$SalePrice*100)  # MAPE: mean absolute percentage error 

In [ ]:
#lasso.val.MSE 
#lasso_val_RMSE # LASSO
#lasso_val_MAPE

#ridge.val.MSE 
#ridge_val_RMSE # Ridge
#ridge_val_MAPE
#Elasticnet_val_RMSE 

results_mod<-c(lasso.val.MSE, lasso_val_RMSE, lasso_val_MAPE, ridge.val.MSE, ridge_val_RMSE, ridge_val_MAPE)
results_mod2<-round(results_mod, digits = 2)
names<- c("lasso.val.MSE", "lasso_val_RMSE", "lasso_val_MAPE", "ridge.val.MSE", "ridge_val_RMSE", "ridge_val_MAPE")
compare= data.frame(names,results_mod2)
compare %>% View()

In [ ]:
test2<-test1[,!(names(test1) %in% drop)]
X_test<-model.matrix(~., test2)[,-1]

Bestmodel<-min(lasso.val.MSE,ridge_val_RMSE)

Optimum_test<- exp(predict(lasso_opt_fit, X_test))

#lasso_test<- exp(predict(lasso_opt_fit, s = penalty_lasso, newx =X_test))
#Lasso_test<- exp(predict(lasso_opt_fit, X_test))
#Elasticnet_test<- exp(predict(elasticnet_opt_fit, X_test))

In [ ]:
my_submission<-tibble(Id=test$Id,SalePrice=Optimum_test)
head(my_submission)
summary(my_submission)

In [ ]:
write.csv(my_submission, file = 'submission.csv',row.names = F)